# Which Bird Are You
Recognising birds by their song using CNNs.

In [ ]:
import keras

## Data loading

In [ ]:
from keras.utils import image_dataset_from_directory
import os

batch_size = 16
image_size = (64, 64)

# If you need train/val split, repeat for both
train_dataset, val_dataset = image_dataset_from_directory(
    "data",
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    subset="both",
    seed=22,
    validation_split=0.2,
)

f = 0.01

train_dataset = train_dataset.take(int(f * len(train_dataset)))
val_dataset = val_dataset.take(int(f * len(val_dataset)))

### Visualisation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

idx_to_bird = {
    i: name for i, name in enumerate(os.listdir("data"))
}

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        idx = int(labels[i])
        plt.title(f"{idx_to_bird[idx]} (#{idx})")
        plt.axis("off")


## Training

In [ ]:
input_shape = (*image_size, 3)
num_classes = len(os.listdir("data"))

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        # keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:
opt = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=opt,
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [ ]:
from tqdm.keras import TqdmCallback

os.makedirs("epochs", exist_ok=True)
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="epochs/model_at_epoch_{epoch}.keras"),
    TqdmCallback(verbose=1),
]

print("Starting training...")
print(f"Model input shape: {model.input_shape}")
print(f"Model output shape: {model.output_shape}")

history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=0,
)

print("Training completed!")

## Results

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over Epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['acc'], label='Training Accuracy')
plt.plot(history.history['val_acc'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over Epochs')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))

# Get one batch from the validation dataset
for images, labels in val_dataset.take(1):
    predictions = model.predict(images)

    for i in range(min(6, len(images))):
        ax = plt.subplot(2, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))

        true_idx = int(labels[i])
        predicted_idx = np.argmax(predictions[i])

        true_label = idx_to_bird[true_idx]
        predicted_label = idx_to_bird[predicted_idx]

        plt.title(f"True: {true_label}\nPred: {predicted_label}")
        plt.axis("off")

plt.tight_layout()
plt.show()